In [3]:
import pandas as pd
import numpy as np
import requests
import datetime
import plotly.graph_objs as go
import plotly.express as px
import locale

In [165]:

def load_chain_data(chain_id):
    chain_url = 'https://indexer-grants-stack.gitcoin.co/data/' + chain_id + '/rounds.json'
    try:
        response = requests.get(chain_url)
        if response.status_code == 200:
            chain_data = response.json()
            rounds = []
            for round in chain_data:
                if round['metadata'] is not None:
                    round_data = {
                        'round_id': round['id'],
                        'name': round['metadata']['name'],
                        'amountUSD': round['amountUSD'],
                        'votes': round['votes'],
                        'description': round['metadata']['description'] if 'description' in round['metadata'] else '',
                        'matchingFundsAvailable': round['metadata']['matchingFunds']['matchingFundsAvailable'] if 'matchingFunds' in round['metadata'] else '',
                        'matchingCap': round['metadata']['matchingFunds']['matchingCap'] if 'matchingFunds' in round['metadata'] else '',
                        'roundStartTime': datetime.datetime.utcfromtimestamp(int(round['roundStartTime'])), # create a datetime object from the timestamp in UTC time
                        'roundEndTime': datetime.datetime.utcfromtimestamp(int(round['roundEndTime']))
                    }
                    rounds.append(round_data)
            df = pd.DataFrame(rounds)
            #start_time = datetime.datetime(2023, 4, 26, 15, 0, 0)
            #end_time = datetime.datetime(2023, 5, 9, 23, 59, 0)
            df = df[(df['votes'] > 0)] #& (df['roundStartTime'] <= start_time) & (df['roundEndTime'] == end_time)]
            return df 
    except: 
        return pd.DataFrame()
    
def load_round_projects_data(round_id):
    # prepare the URLs
    projects_url = 'https://indexer-grants-stack.gitcoin.co/data/1/rounds/' + round_id + '/projects.json'
    
    try:
        # download the Projects JSON data from the URL
        response = requests.get(projects_url)
        if response.status_code == 200:
            projects_data = response.json()

        # Extract the relevant data from each project
        projects = []
        for project in projects_data:
            project_data = {
                'id': project['id'],
                'title': project['metadata']['application']['project']['title'],
                'description': project['metadata']['application']['project']['description'],
                'status': project['status'],
                'amountUSD': project['amountUSD'],
                'votes': project['votes'],
                'uniqueContributors': project['uniqueContributors']
            }
            projects.append(project_data)
        # Create a DataFrame from the extracted data
        dfp = pd.DataFrame(projects)
        # Reorder the columns to match the desired order and rename column id to project_id
        dfp = dfp[['id', 'title', 'description', 'status', 'amountUSD', 'votes', 'uniqueContributors']]
        dfp = dfp.rename(columns={'id': 'project_id'})
        # Filter to only approved projects
        #dfp = dfp[dfp['status'] == 'APPROVED']
        return dfp
    except:
        return pd.DataFrame()
    
def load_round_votes_data(round_id):
    votes_url = 'https://indexer-grants-stack.gitcoin.co/data/1/rounds/' + round_id + '/votes.json'
    try:
        # download the Votes JSON data from the URL
        response = requests.get(votes_url)
        if response.status_code == 200:
            votes_data = response.json()
        df = pd.DataFrame(votes_data)
        return df
    except:
        return pd.DataFrame()
    

def load_all_votes_data(chain_data):
    dfv_all = pd.DataFrame()
    for round_id in chain_data['round_id']:
        if round_id == '0x0000000000000000000000000000000000000000':
            print("*******DUDE Skipping round 0")
            continue
        #print("Loading Round with ID: " + round_id)
        dfv = load_round_votes_data(round_id)
        dfp = load_round_projects_data(round_id)
        dfv = pd.merge(dfv, dfp[['project_id', 'title', 'status']], how='left', left_on='projectId', right_on='project_id')
        # CHECK IF chain_data has columns roun_type and matching_amount
        if 'round_type' not in chain_data.columns:
            chain_data['round_type'] = ''
        if 'matching_amount' not in chain_data.columns:
            chain_data['matching_amount'] = 0
        if 'round_name' not in chain_data.columns:
            #rename name column to round_name
            chain_data = chain_data.rename(columns={'name': 'round_name'})
        dfv = pd.merge(dfv, chain_data[['round_id', 'round_name', 'round_type', 'matching_amount']], how='left', left_on='roundId', right_on='round_id')
        dfv_all = pd.concat([dfv_all, dfv])
    return dfv_all

def load_passport_data():
    url = 'https://indexer-grants-stack.gitcoin.co/data/passport_scores.json'
    try:
        response = requests.get(url)
        response.raise_for_status()
        passports_data = response.json()
        if not passports_data:
            raise ValueError('Passport data is empty')
        else:
            print("Passport data loaded successfully!")
        passports = []
        for passport in passports_data:
            data = {
                'address': passport.get('address', ''),
                'score': passport.get('score', ''),
                'status': passport.get('status', ''),
                'last_score_timestamp': passport.get('last_score_timestamp', ''),
                'evidence_type': passport['evidence'].get('type', '') if passport.get('evidence') else '',
                'evidence_success': passport['evidence'].get('success', '') if passport.get('evidence') else '',
                'evidence_rawScore': passport['evidence'].get('rawScore', '') if passport.get('evidence') else '',
                'evidence_threshold': passport['evidence'].get('threshold', '') if passport.get('evidence') else '',
                'error': passport.get('error', ''),
            }
            passports.append(data)
        df = pd.DataFrame(passports)
        return df
    except Exception as e:
        print("Error:", e)
        return pd.DataFrame()

In [ ]:
# RUN ONCE
chain_id = '1'
df = load_chain_data(chain_id)
df_all = load_all_votes_data(df)
# save df_all to a csv
df_all.to_csv('beta_votes_all.csv', index=False)
df = load_passport_data()
# Save to CSV
df.to_csv('passport_data.csv', index=False)

In [ ]:
# RUN ON BOOT UP
df.read_csv('beta_votes_all.csv')
df.read_csv('passport_data.csv')

In [11]:
### ADJUST PARAMETERS HERE
min_passport_score = 15.0
min_amountUSD = .90
power = 2 # change to 2
matching_cap_ratio = 0.10
matching_pool = 350000


def get_matching(votes_data):
    votes_data = get_voting_weights(votes_data)
    votes_data = votes_data[votes_data['voting_success'] == True]
    project_data = votes_data.groupby(['title']).agg({'amountUSD': 'sum', 'sqrt_amountUSD':'sum', 'id': 'sum', 'voter': 'nunique'}).reset_index().sort_values('amountUSD', ascending=False)
    project_data = project_data.rename(columns={'sqrt_amountUSD': 'sum_sqrt_amountUSD'})
    project_data['squared_sum_sqrts'] = project_data['sum_sqrt_amountUSD'] ** power
    project_data['squared_sum_sqrts_minus_donations'] = project_data['squared_sum_sqrts'] - project_data['amountUSD'] #minus green squares
    project_data['matching_ratio'] = project_data['squared_sum_sqrts_minus_donations'] / project_data['squared_sum_sqrts_minus_donations'].sum()
    project_data['matching_amount'] = project_data['matching_ratio'] * matching_pool
    if (project_data['matching_ratio'] > matching_cap_ratio).any():
        project_data = check_matching_cap(project_data)
    round_saturation = (project_data['squared_sum_sqrts_minus_donations'].sum() / matching_pool) * 100
    print("Round Saturation: " + '{:,.2f}'.format(round_saturation) + "%") 
    return project_data

def get_voting_weights(votes_data):
    votes_data_with_weights = votes_data#.groupby(['voter', 'title']).agg({'amountUSD': 'sum', 'id': 'nunique'}).reset_index().sort_values('amountUSD', ascending=False)
    # flag where min_amount_usd threshold not met 
    votes_data_with_weights['amountUSD_above_min'] = np.where(votes_data_with_weights['amountUSD'] > min_amountUSD, True, False)
    votes_data_with_weights['passport_score'] = votes_data_with_weights['voter'].str.lower().map(df_pp.set_index('address')['evidence_rawScore'])
    votes_data_with_weights['passport_success'] = np.where(votes_data_with_weights['passport_score'] >= min_passport_score, True, False)
    votes_data_with_weights['voting_success'] = (votes_data_with_weights['amountUSD_above_min'] & votes_data_with_weights['passport_success'])
    votes_data_with_weights['sqrt_amountUSD'] = (votes_data_with_weights['amountUSD']) ** (1/power)
    return votes_data_with_weights

def check_matching_cap(project_data):
    matching_cap_amount = matching_cap_ratio * matching_pool
    while True:
        # Calculate over cap amounts and total uncapped fund
        over_cap = np.maximum(0, project_data['matching_amount'] - matching_cap_amount)
        total_fund_for_not_capped = project_data['matching_amount'][project_data['matching_amount'] < matching_cap_amount].sum()

        # Reduce matching amounts that are over cap
        project_data.loc[project_data['matching_amount'] > matching_cap_amount, 'matching_amount'] = matching_cap_amount
               # Calculate remainder percent and update matching amounts
        if total_fund_for_not_capped > 0:
            remainder_percent = over_cap.sum() / total_fund_for_not_capped
            project_data.loc[project_data['matching_amount'] < matching_cap_amount, 'matching_amount'] *= (1 + remainder_percent)
        else:
            break  # Exit loop when there's no fund left for not capped projects
        
        # Ensure that updates didn't push any projects over the cap
        over_cap_after_update = np.maximum(0, project_data['matching_amount'] - matching_cap_amount)
        if not over_cap_after_update.sum() > 0:
            break  # Exit loop when there's no amount over cap
    project_data['matching_ratio'] = project_data['matching_amount'] / matching_pool
    return project_data

In [9]:
# filter df_all to only include round_id that is 0x274554EB289004e15A7679123901B7F070dDa0fa
df_zk = df_all[df_all['round_id'] == '0x421510312C40486965767be5Ea603Aa8a5707983']
#df_test = get_voting_weights(df_zk)
# save to csv
#df_test.to_csv('beta_votes_oss_new.csv', index=False)
df_test = get_matching(df_zk)
df_test = df_test.sort_values('matching_ratio', ascending=False)
# display without scientific notation 
pd.options.display.float_format = '{:.2f}'.format
# print dataframe matching_amount and matching_ratio sums
print("matching_amount sum: " + '{:,.2f}'.format(df_test['matching_amount'].sum()))
print("matching_ratio sum: " + '{:,.2f}'.format(df_test['matching_ratio'].sum()))
#display(df_test)
# filter to title, matching_amount, matching_ratio
df_test = df_test[['title', 'matching_amount', 'matching_ratio']]
# display all rows
pd.set_option('display.max_rows', None)
# display dataframe
display(df_test.reset_index(drop=True))

Round Saturation: 116.51%
matching_amount sum: 350,000.00
matching_ratio sum: 1.00


/var/folders/vc/4gnlrkhx1bqgc_p_rhn78pxh0000gn/T/ipykernel_871/883549841.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  votes_data_with_weights['amountUSD_above_min'] = np.where(votes_data_with_weights['amountUSD'] > min_amountUSD, True, False)
/var/folders/vc/4gnlrkhx1bqgc_p_rhn78pxh0000gn/T/ipykernel_871/883549841.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  votes_data_with_weights['passport_score'] = votes_data_with_weights['voter'].str.lower().map(df_pp.set_index('address')['evidence_rawSc

,title,matching_amount,matching_ratio
0,Atlantis,35000.00,0.10
1,WarmOFF - CO₂ removal project,35000.00,0.10
2,$Earth - Solarpunk Dao,32091.31,0.09
3,Block2030,25267.45,0.07
4,Treegens - The Most Transparent & Rewarding Tr...,24039.83,0.07
5,Mini Meadows,22453.27,0.06
6,diosdao.xyz by Mycelia : Indigenous Public Goo...,22069.17,0.06
7,Treejer Protocol,20397.04,0.06
8,WaterDAO,10146.52,0.03
9,Web3beach,9934.32,0.03


In [12]:
# filter df_all to only include round_id that is 0x274554EB289004e15A7679123901B7F070dDa0fa
df_zk = df_all[df_all['round_id'] == '0x421510312C40486965767be5Ea603Aa8a5707983']
#df_test = get_voting_weights(df_zk)
# save to csv
#df_test.to_csv('beta_votes_oss_new.csv', index=False)
df_test = get_matching(df_zk)
df_test = df_test.sort_values('matching_ratio', ascending=False)
# display without scientific notation 
pd.options.display.float_format = '{:.2f}'.format
# print dataframe matching_amount and matching_ratio sums
print("matching_amount sum: " + '{:,.2f}'.format(df_test['matching_amount'].sum()))
print("matching_ratio sum: " + '{:,.2f}'.format(df_test['matching_ratio'].sum()))
#display(df_test)
# filter to title, matching_amount, matching_ratio
df_test = df_test[['title', 'matching_amount', 'matching_ratio']]
# display all rows
pd.set_option('display.max_rows', None)
# display dataframe
display(df_test.reset_index(drop=True))

Round Saturation: 123.01%
matching_amount sum: 350,000.00
matching_ratio sum: 1.00


/var/folders/vc/4gnlrkhx1bqgc_p_rhn78pxh0000gn/T/ipykernel_871/2497222026.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  votes_data_with_weights['amountUSD_above_min'] = np.where(votes_data_with_weights['amountUSD'] > min_amountUSD, True, False)
/var/folders/vc/4gnlrkhx1bqgc_p_rhn78pxh0000gn/T/ipykernel_871/2497222026.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  votes_data_with_weights['passport_score'] = votes_data_with_weights['voter'].str.lower().map(df_pp.set_index('address')['evidence_raw

,title,matching_amount,matching_ratio
0,WarmOFF - CO₂ removal project,35000.00,0.10
1,Atlantis,35000.00,0.10
2,$Earth - Solarpunk Dao,31492.39,0.09
3,Treegens - The Most Transparent & Rewarding Tr...,26071.08,0.07
4,Block2030,24074.07,0.07
5,diosdao.xyz by Mycelia : Indigenous Public Goo...,21577.05,0.06
6,Mini Meadows,21124.42,0.06
7,Treejer Protocol,19189.88,0.05
8,Web3beach,10257.95,0.03
9,WaterDAO,9546.02,0.03
